In [19]:
#This program looks at AAPL options data and calculates implied volatility for selected OTM, ATM, and ITM options using the Black-Scholes model.
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm

In [21]:
# Fetch Apple Inc. (AAPL) options data
ticker = 'AAPL'
stock = yf.Ticker(ticker)
#
# Get the next available expiration date (~3 months out)
expiration_dates = stock.options
expiration_date = expiration_dates[1]  # Choosing the second expiration date
option_chain = stock.option_chain(expiration_date)
#
# Extract call option data
calls = option_chain.calls
#
# Get the current stock price using the updated indexing method
current_price = stock.history(period="1d")['Close'].iloc[0]  # Use .iloc for positional indexing
#
print(f"Current stock price of AAPL: {current_price}")

Current stock price of AAPL: 233.0


In [23]:
# Select options: 2 out-of-the-money (OTM), 1 at-the-money (ATM), and 2 in-the-money (ITM)
OTM_options = calls[calls['strike'] > current_price].head(2)
ATM_option = calls.iloc[(calls['strike'] - current_price).abs().argsort()[:1]]
ITM_options = calls[calls['strike'] < current_price].head(2)
# Combine all selected options into one dataframe
selected_options = pd.concat([OTM_options, ATM_option, ITM_options])
print("Selected Options:\n", selected_options[['contractSymbol', 'strike', 'lastPrice']])

Selected Options:
          contractSymbol  strike  lastPrice
27  AAPL241011C00235000   235.0       2.65
28  AAPL241011C00237500   237.5       1.65
26  AAPL241011C00232500   232.5       3.95
0   AAPL241011C00100000   100.0     128.52
1   AAPL241011C00120000   120.0     103.30


In [25]:
# Function to calculate Black-Scholes theoretical option price
def bsval(S, K, T, rf, sigma, type):
    # Ensure sigma is positive
    if sigma <= 0:
        return np.nan
    try:
        d1 = (np.log(S / K) + (rf + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
    except (ZeroDivisionError, ValueError, OverflowError) as e:
        print("Encountered an error in bsval calculation:", e)
        return np.nan  
    if type == 1:
        # Call option price calculation
        return S * norm.cdf(d1) - K * np.exp(-rf * T) * norm.cdf(d2)
    else:
        # Put option price calculation
        return K * np.exp(-rf * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [15]:
# Function to calculate implied volatility for each option
def calculate_implied_volatility(price, S, K, T, rf, type):
    guess1 = 0.5
    val1 = bsval(S, K, T, rf, guess1, type)
    guess2 = 0.075
    val2 = bsval(S, K, T, rf, guess2, type)
    check = abs(price - val2)
    iteration = 0
    max_iterations = 100  # Set a limit to avoid infinite loops
    while check > 0.01 and iteration < max_iterations:
        iteration += 1
        # Handle potential division by zero
        if val2 == val1:
            print("Division by zero detected. Adjusting guesses.")
            guess2 += 0.01  # Slight adjustment to avoid division by zero
            val2 = bsval(S, K, T, rf, guess2, type)
            continue  
        try:
            guessnew = guess2 - ((guess2 - guess1) / (val2 - val1)) * (val2 - price)
        except ZeroDivisionError:
            print("ZeroDivisionError detected during calculation. Adjusting guess.")
            guessnew = guess2 + 0.01  # Adjust to continue   
        guess1 = guess2
        guess2 = guessnew
        val1 = val2
        val2 = bsval(S, K, T, rf, guess2, type) 
        # Recalculate the check value
        check = abs(price - val2)
    if iteration >= max_iterations:
        print(f"Did not converge within {max_iterations} iterations for option price =", price)
        return np.nan 
    implied_vol = guess2 * 100
    implied_vol = np.round(implied_vol, 2)
    return implied_vol

In [17]:
# Apply the implied volatility calculation to each selected option
T = 0.25 
rf = 0.05  
type = 1  
#
# Calculate implied volatility for each option and add it to the dataframe
selected_options['Implied_Volatility'] = selected_options.apply(
    lambda row: calculate_implied_volatility(row['lastPrice'], current_price, row['strike'], T, rf, type),
    axis=1)
print("Selected Options with Implied Volatility")
print(selected_options[['contractSymbol', 'strike', 'lastPrice', 'Implied_Volatility']])

Selected Options with Implied Volatility
         contractSymbol  strike  lastPrice  Implied_Volatility
27  AAPL241011C00235000   235.0       2.65                4.67
28  AAPL241011C00237500   237.5       1.65                5.03
26  AAPL241011C00232500   232.5       3.95                3.82
0   AAPL241011C00100000   100.0     128.52           -84907.33
1   AAPL241011C00120000   120.0     103.30           -11013.60


In [29]:
# Observations
# The OTM options show implied volatility in reason.
# The ITM options have extreme negative values, suggesting calculation issues.
# Implied volatility is more reliable for OTM than ITM options using this method?